# Week1_Library 과제

### Q1. Library 와 Framework 의 차이 간단하게 서술하시오. (100자 내외)

프레임워크는 자체적으로 흐름을 가지고 있어 그 안에서 프로그래머가 코드를 작성하는 것이라면, 라이브러리는 프로그래머가 전체적인 흐름을 가지고  원하는 기능을 구현하고 싶을 때 가져다 사용할 수 있다.

### Q2. 딥러닝과 머신러닝의 관계 및 특징, 차이 간단하게 서술하시오. (200자 내외)

**<딥러닝과 머신러닝의 관계 및 특징>** <br>
딥러닝은 머신러닝 범주 안에 들어가는 개념이다. 
머신러닝은 통계학에서 유래된 머신러닝 알고리즘이 많으며 통계학과 컴퓨터 과학 분야가 상호 작용하면서 발전하고 있다. 대표적인 머신러닝 라이브러리는 사이킷런. 딥러닝은 많은 머신러닝 알고리즘 중에 인공 신경망을 기반으로 한 방법들을 통칭하는 개념이다.대표적인 딥러닝 라이브러리로는 텐서플로, 파이토치가 있다. 

**<딥러닝과 머신러닝의 차이>**<br>
딥러닝은 처음부터 기계가 학습하는 구조라면 머신러닝은 인간이 데이터 분석의 힌트를 알려준 후 분석하는 학습방법이다.딥러닝은 일반적으로 인간이 개입해야 할 상황이 적은 반면 머신러닝은 상대적으로 컴퓨팅 능력을 덜 필요로 한다. 딥러닝은 머신러닝이 쉽게 할 수 없는 이미지, 비디오, 비정형 데이터를 분석할 수 있다.

### Q3. 아래의 코드에 주석 달기.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms

device = 'cuda' if torch.cuda.is_available() else 'cpu' # GPU있으면 사용한다는 뜻.
torch.manual_seed(45) # cpu 연산 무작위 고정. pytorch에서 사용하는 random seed 고정 함수. 현재 코드에서는 random seed를 45로 고정함.
if device == 'cuda': # gpu 사용 가능하다면
    torch.cuda.manual_seed_all(45) # multi-GPU 연산 고정
print(device + " is available") # 현재 사용 가능한 device 출력. gpu면 'cuda', cpu면 'cpu' 츌력

In [ ]:
learning_rate = 0.001 # 한 번 학습할 때 학습할 양, 학습률이 너무 크면 발산할 수 있고, 너무 작으면 속도가 느려지고 계산량이 많아짐.
# 초기값으로 0.1, 0.01, 0.001 선호
batch_size = 100 # parameter를 업데이트 할 때 사용할 *데이터 개수*, 100개의 문제가 있을 때 20개씩 풀고 채점한다면 batch size는 20. batch크기만큼 데이터를 활용해 모델이 예측한 값과 실제 정답 간의 오차를 계산하여 optimizer가 parameter를 업데이트함.
num_classes = 10 # 클래스 개수, MNIST 데이터셋은 0~9까지의 숫자의 손글씨 이미지 데이터이므로 클래스개수는 10개이다.
epochs = 5 # 전체 데이터셋을 학습한 횟수, 문제집 한 권을 몇 회 풀었는지. 문제집을 5번 풀었다면 epoch는 5

In [ ]:

train_set = torchvision.datasets.MNIST( # torchvision 라이브러리에 포함된 MNIST 데이터셋을 가져온다.
    root = './data/MNIST', # 다운로드 경로 지정
    train = True,# 테스트용 데이터를 가져올지, 학습용 데이터를 가져올지 표시. True이므로 학습용 데이터 가져옴.
    download = True, # True이면 MNIST 데이터가 없으면 다운 받는다. 
    transform = transfroms.Compose([ # 어떤 형태로 데이터를 불러올 것인가를 설정. 일반 이미지는 0-255 범위의 값을 갖고, (H,W,C)형태를 갖는 반면, pytorch는 0-1 범위 값을 가지고 (C,H,W)형태를 가짐.
        transfroms.ToTensor() # transforms.ToTensor()를 넣어 일반 이미지(PIL image)를 pytorch tensor로 변환한다.
    ]) 
)

test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False, # 테스트용 데이터를 가져올지, 학습용 데이터를 가져올지 표시. False이므로 테스트 데이터 가져옴.
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)

In [ ]:
# 위에서 불러온 데이터셋 객체로 데이터 로더 객체를 만든다. 이후 이 객체에서 데이터를 뽑아 쓴다.
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size) # 로드할 dataset은 train_set으로, 배치사이즈는 위에서 정의한 배치사이즈로 설정함.
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)# 로드할 dataset은 test_set으로, 배치사이즈는 위에서 정의한 배치사이즈로 설정함.

# input size를 알기 위해서
examples = enumerate(train_set) 
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape # input size

In [ ]:
# layer 정의, 네트워크 내부에서 사용할 구조를 만든다.
class ConvNet(nn.Module):
    def __init__(self): 
        super(ConvNet, self).__init__() 
        
        # input size = 28x28
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # input channel=1, filter=10, kernel size=5, (기본) zero padding=0, stribe=1
        # ((W-K+2P)/S)+1 공식으로 인해 ((28-5+0)/1)+1=24 -> 24x24로 변환
        # maxpooling하면 12x12
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) # input channel = 1, filter = 10, kernel size = 5, (기본)  zero padding = 0, stribe = 1
        # ((12-5+0)/1)+1=8 -> 8x8로 변환
        # maxpooling하면 4x4
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) # 랜덤하게 뉴런을 종료해서 학습을 방해해 학습이 학습용 데이터에 치우치는 현상을 막기 위해 사용
        self.mp = nn.MaxPool2d(2) # 오버피팅을 방지하고, 연산에 들어가는 자원을 줄이기 위해 maxpolling
        self.fc1 = nn.Linear(320,100) # 4x4x20 vector로 flat한 것을 100개의 출력으로 변경
        self.fc2 = nn.Linear(100,10)  # 100개의 출력을 10개의 출력으로 변경


    def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) # convolution layer 1번에 relu를 씌우고 maxpool, 결과값은 12x12x10
        x = F.relu(self.mp(self.conv2(x))) # convolution layer 2번에 relu를 씌우고 maxpool, 결과값은 4x4x20
        x = self.drop2D(x) 
        x = x.view(x.size(0), -1) # flat
        x = self.fc1(x) # fc1 레이어에 삽입
        x = self.fc2(x) # fc2 레이어에 삽입
        return F.log_softmax(x) # fully-connected layer에 넣고 logsoftmax 적용

In [ ]:

model = ConvNet().to(device) # CNN instance 생성
# Cost Function과 Optimizer 선택 
criterion = nn.CrossEntropyLoss().to(device) # Cost function 정의
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # Optimizer 정의

In [ ]:
# epochs수만큼 반복, for문이 한 번 돌때(1 epoch 학습)마다 batch_size만큼의 데이터를 꺼내서 학습시킴. 
for epoch in range(epochs): 
    avg_cost = 0 # train set에서 batch_size만큼 돌릴 것이고, avg_cost에 loss를 저장할 것이다.
    for data, target in train_loader: # batch_size 단위로 꺼내온다.
        data = data.to(device) # 연산하고자 하는 Tensor를 사용하는 device에 올려준다. 
        target = target.to(device)
        optimizer.zero_grad() # optimizer를 하기 전 모든 model의 gradient 값을 0으로 초기화
        hypothesis = model(data) # forward propagation 수행, 모델을 forward pass해 결과값 저장
        cost = criterion(hypothesis, target) # 위에서 정의한 loss 함수로 output과 target의 loss 계산
        cost.backward() # backward 함수를 호출해 gradient 계산
        optimizer.step() # 모델의 학습 parmeter 값 갱신, grad descent
        avg_cost += cost / len(train_loader) # loss 값을 변수에 누적하고 train_loader의 개수로 나눔 = 평균
    print('[Epoch: {:>4}]  cost = {:>.9}'.format(epoch + 1, avg_cost))

In [ ]:
# test set 이용해서 test
model.eval() # evaluate mode로 전환 dropout 이나 batch_normalization 해제
with torch.no_grad(): # test에서는 grad를 계산할 필요가 없으므로 grad 해제 
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device) # 연산하고자 하는 Tensor를 사용하는 device에 올려준다.
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] # 출력이 클래스 각각에 대한 값으로 나타나기 때문에, 가장 높은 값을 갖는 인덱스를 추출
        total += len(target) # 전체 클래스 개수 
        correct += (preds==target).sum().item() # 예측값과 실제값이 같은지 비교
        
        
    print('Test Accuracy: ', 100.*correct/total, '%')
     

## 첫 정규세션 들으시느라 고생 많으셨습니다.

고생하셨습니다!!